# download


    !curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt --output input.txt

    !wget -O input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [5]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    print(len(text))

1115394


In [7]:
# unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars), vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65


In [29]:
# tokenizer - typically subword level:
# tiktoken bpe from openai used in gpts, and sentencepiece from google, for llama, mistral
# many possibilities for tokenization, different schema
# trade off code book vocabulary size and encoded sequence length:
# small vocabularies and long sequence length, or
# large vocabularies and short sequence length 

[24912, 2375, 220]


In [ ]:
# !pip install tiktoken
import tiktoken
enc = tiktoken.get_encoding("o200k_base")
enc = tiktoken.encoding_for_model("gpt-4o")
assert enc.decode(enc.encode("hello world")) == "hello world"
print(enc.encode('hello world '))

# !pip install sentencepiece
# !pip install protobuf
# https://github.com/google/sentencepiece
import sentencepiece as spm
s = spm.SentencePieceProcessor(model_file='spm.model)
print(s.encode('hello world'))

In [19]:

# create simple mapping between integer and chars
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s :  [stoi[c] for c in s] # encode string to list of ints
decode = lambda l :  ''.join([itos[i] for i in l]) # decode list of ints to string

print(encode("hello how are you"))
print(decode(encode("hello how are you")))

[46, 43, 50, 50, 53, 1, 46, 53, 61, 1, 39, 56, 43, 1, 63, 53, 59]
hello how are you


In [39]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:10])
print(decode(encode(text[:20])))

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])
First Citizen:
Befor


In [44]:
# train test split
n = int(len(data) * 0.9) # first 90% will be train
train_data = data[:n]
val_data = data[n:]
print(len(train_data), len(val_data))

1003854 111540


In [46]:
# we work only with chunk of token dataset
# with additional parameter block_size
# which is longest context 

block_size = 8
x = train_data[:block_size] # take a trunk
y = train_data[1:block_size+1] # take the moving y

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [81]:
# add batch function to allow batch processing

torch.manual_seed(1337)
batch_size = 4 # how many sequences with be processed in parallel
block_size = 8 # what is the matximum context length for prediction

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return  x, y

xb, yb = get_batch('train')

print('inputs', xb.shape, yb.shape)
print(xb)
print(yb)

print('-------')
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

## ?? 
#  this may make training less effective, as always same t is used in one step
#  if t is different among the batch, may be good
#  or gradient accumulation with multiple steps, from different t 


inputs torch.Size([4, 8]) torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53, 56, 1

In [62]:
xb

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

In [64]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

In [78]:
# implement bigram language model

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits
        # for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):

        # idx and targes are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # B, T, C [4, 8, 65]

        if targets == None:
            loss = None
        else:
            # reshape to match torch ce loss input format
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)        
            loss = F.cross_entropy(logits, targets) 
        
        return logits, loss
    
    def generate(self, idx, max_new_token):
        # idx is (B, T) array of indices in current context
        for _ in range(max_new_token):
            # get the prediction
            logits, loss = self(idx) # by default call forward
            # focus only on the last time step
            logits = logits[:,-1,:] # the last T tells next token, becomes (B, C)
            # apply softmax to get probs
            probs = F.softmax(logits, dim = -1) # norm along channel, (B, C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx


    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

print(vocab_size)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print('sample idx:', idx)
print('random model:', decode(m.generate(idx, max_new_token=100)[0].tolist()))


65
tensor(4.6691, grad_fn=<NllLossBackward0>)
sample idx: tensor([[0]])
random model: 
!zXNncOx;Y-ua,iopHr?CBpD:OOEfUVXEEErpQWolyntZoM-AH,uIXvV;GduRO
R-,yU3KRNAatvLDE-ib3.quhv.hOL.R3hCSG.


In [91]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)
batch_size = 64
for steps in range(100000):
    
    # sample a batch
    xb, yb = get_batch('train')

    # get loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True) # set to none = True faster
    loss.backward()
    optimizer.step()
    if steps%10000 == 0:
        print(loss.item()) 

print(decode(m.generate(idx=torch.zeros((1,1), dtype = torch.long),
                        max_new_token=300
                        )[0].tolist()))

2.494992256164551
2.4298572540283203
2.418480634689331
2.441776752471924
2.42606520652771
2.5111498832702637
2.5172455310821533
2.4108800888061523
2.4815258979797363


KeyboardInterrupt: 